In [1]:
from faker import Faker
from random import randint
import random
import pandas as pd

faker = Faker('pt_BR')
Faker.seed(0)

# Clients Table
First, I'm gonna makeup some **Client Claims**, as this would be the first step: The Client with a problem would get in touch with a dealer and claim that some problem is ocurring in his vehicle

In [3]:
# Define the vehicles and their weights
vehicles = {
    "Dolphin": "BYD001", 
    "Seal": "BYD002",
    "Yuan": "BYD003",
    "Han": "BYD004",
    "Tan": "BYD005"
}

vehicles_weights = {
    "Dolphin": 20,
    "Seal": 2,
    "Yuan": 6,
    "Han": 3,
    "Tan": 3
}

# Define common car problems
# Define the possible problems and their weights
problems = {
    "Battery degradation": 20,
    "Charging issues": 18,
    "Electric motor failure": 15,
    "Range anxiety": 12,
    "Battery management system failure": 14,
    "Overheating of battery": 10,
    "Electrical system faults": 11,
    "Regenerative braking issues": 9,
    "Inverter problems": 8,
    "Cooling system issues": 7,
    "Charging port issues": 6,
    "Software glitches": 5,
    "Wiring problems": 4,
    "Vehicle control system failures": 3,
    "Battery overheating": 2,
    "Inconsistent charging": 2,
    "Climate control system issues": 1,
    "Connectivity issues": 1,
    "Suspension problems": 3,
    "Windshield replacement": 5, 
    "Brake system maintenance": 4, 
    "Tire replacement": 4,  
    "Fluid top-up": 2,  
    "Cabin filter replacement": 3 
}


    
def select_car():
    cars = list(vehicles.keys())
    weights = [vehicles_weights[car] for car in cars]
    return random.choices(cars, weights=weights, k=1)[0]

def select_problem():
    problems_list = list(problems.keys())
    weights = [problems[problem] for problem in problems_list]
    return random.choices(problems_list, weights=weights, k=1)[0]

def create_request_table(x):

    data = pd.DataFrame()
    
    for i in range(1,x+1):
        #client_name = f"{faker.first_name()} {faker.last_name()}"
        claim = select_problem()
        car = select_car()
        
        data.loc[i,'CLIENT_ID'] = i
        data.loc[i,'CLIENT_FIRST_NAME'] = faker.first_name()
        data.loc[i,'CLIENT_LAST_NAME'] = faker.last_name()
        data.loc[i,'CAR'] = car
        data.loc[i,'CLAIM'] = claim
        data.loc[i,'dealerId'] = f"{(randint(1,50)):06}"
        data.loc[i, 'dealerOrderId'] = f"{(i):06}" #Added here so I can retrieve

    return data

In [4]:
request_table = create_request_table(5)
#request_table.set_index('dealerOrderId', inplace=True)

In [5]:
request_table

,CLIENT_ID,CLIENT_FIRST_NAME,CLIENT_LAST_NAME,CAR,CLAIM,dealerId,dealerOrderId
1,1.0,Helena,Teixeira,Han,Suspension problems,000009,000001
2,2.0,Laura,Aparecida,Dolphin,Battery management system failure,000006,000002
3,3.0,Valentim,Moraes,Dolphin,Battery management system failure,000039,000003
4,4.0,Maria Vitória,Guerra,Dolphin,Charging issues,000026,000004
5,5.0,Ana Júlia,Mendes,Dolphin,Charging issues,000050,000005


# Repair Order Table
Now that I already have some information about the Client, the Dealer will open the main request. 

In [7]:
def generate_main_table(base_table):
    data = pd.DataFrame()

    for i, row in base_table.iterrows():
        
        createDate = faker.date_this_year()
        updateDate = faker.date_between(start_date=createDate)

        dominio = random.choice(["@gmail.com","@outlook.com","@yahoo.com.br","uol.com.br"])
        email = f"{row['CLIENT_LAST_NAME']}.{row['CLIENT_FIRST_NAME']}@{dominio}"


        data.loc[i, 'dealerOrderId'] = row['dealerOrderId']
        data.loc[i, 'dealerId'] = row['dealerId']
        data.loc[i, 'dealerOrderNo'] = randint(20,120)    #Inconsistent for now, it isnt tracking each dealer's order count
        data.loc[i, 'createBy'] = '9999999999'
        data.loc[i, 'createName'] = 'Name'
        #data.loc[i, 'createDate'] = createDate
        data.loc[i, 'updateBy'] = '9999999988'
        data.loc[i, 'updateName'] = 'Name'
        #data.loc[i, 'updateDate'] = updateDate
        data.loc[i, 'repairType'] = random.choice(['Warranty', 'Non-Warranty'])
        data.loc[i, 'orderStatus'] = random.choice(['Pending', 'In Progress', 'Completed'])
        data.loc[i, 'orderCreateDate'] = createDate
        data.loc[i, 'orderLastBalanceDate'] = updateDate
        data.loc[i, 'vin'] = faker.unique.license_plate()
        data.loc[i, 'custName'] = f"{row['CLIENT_FIRST_NAME']} {row['CLIENT_LAST_NAME']}"
        data.loc[i, 'custEmail'] = email
        data.loc[i, 'deliverer'] = faker.name()
        data.loc[i, 'delivererMobile'] = faker.phone_number()
        data.loc[i, 'powerMileage'] = faker.random_int(min=0, max=50000)
        data.loc[i, 'totalMileage'] = faker.random_int(min=0, max=100000)
        data.loc[i, 'createClaimFlag'] = random.choice(['Y', 'N'])
        data.loc[i, 'claimNo'] = faker.unique.uuid4() if data.loc[i, 'createClaimFlag'] == 'Y' else None
        data.loc[i, 'activityFlag'] = random.choice(['Y', 'N'])
        data.loc[i, 'activityCode'] = faker.unique.uuid4() if data.loc[i, 'activityFlag'] == 'Y' else None
        data.loc[i, 'troubleDesc'] = row['CLAIM']
        data.loc[i, 'deliverProblem'] = None
        data.loc[i, 'checkResult'] = None
        data.loc[i, 'isSecstore'] = random.choice(['Y', 'N'])
        data.loc[i, 'secstoreId'] = faker.unique.uuid4() if data.loc[i, 'isSecstore'] == 'Y' else None

    return data

In [8]:
REPAIR_ORDER_MAIN = generate_main_table(request_table)

In [9]:
REPAIR_ORDER_MAIN

,dealerOrderId,dealerId,dealerOrderNo,createBy,createName,updateBy,updateName,repairType,orderStatus,orderCreateDate,...,totalMileage,createClaimFlag,claimNo,activityFlag,activityCode,troubleDesc,deliverProblem,checkResult,isSecstore,secstoreId
1,000001,000009,107.0,9999999999,Name,9999999988,Name,Warranty,Completed,2024-05-19,...,83941.0,N,None,N,None,Suspension problems,None,None,N,None
2,000002,000006,50.0,9999999999,Name,9999999988,Name,Warranty,Pending,2024-03-20,...,71170.0,Y,fe43c49e-1498-48d1-9759-edc372ae2244,N,None,Battery management system failure,None,None,Y,ff7b118e-8208-45d6-a005-b86051ef1922
3,000003,000039,47.0,9999999999,Name,9999999988,Name,Non-Warranty,Completed,2024-07-09,...,24823.0,N,None,N,None,Battery management system failure,None,None,N,None
4,000004,000026,106.0,9999999999,Name,9999999988,Name,Warranty,In Progress,2024-03-13,...,59056.0,Y,ff0ac0f1-a425-499a-a905-d7507e1ea9c5,Y,cb175a5a-fb82-460d-aabc-a8d0b341facd,Charging issues,None,None,N,None
5,000005,000050,97.0,9999999999,Name,9999999988,Name,Warranty,Pending,2024-05-18,...,75217.0,N,None,N,None,Charging issues,None,None,N,None


# Dealer Info Table
Now we will generate 50 dealers for the Dealer Info table

In [11]:
regions_and_states = {
    'Brazil North': {
        'Amazonas': ['Manaus', 'Parintins', 'Itacoatiara'],
        'Pará': ['Belém', 'Ananindeua']
    },
    'Brazil Northeast': {
        'Bahia': ['Salvador'],
        'Ceará': ['Fortaleza', 'Juazeiro do Norte', 'Sobral']
    },
    'Brazil Central-West': {
        'Goiás': ['Goiânia', 'Anápolis'],
        'Mato Grosso': ['Cuiabá','Rondonópolis']
    },
    'Brazil Southeast': {
        'São Paulo': ['São Paulo', 'Campinas', 'Santos', 'São José dos Campos'],
        'Rio de Janeiro': ['Rio de Janeiro', 'Niterói']
    },
    'Brazil South': {
        'Paraná': ['Curitiba', 'Londrina'],
        'Rio Grande do Sul': ['Porto Alegre', 'Caxias do Sul', 'Pelotas']
    }
}

def generate_dealers(x):
    data = pd.DataFrame()

    for i in range(x):

        region = random.choice(list(regions_and_states.keys()))
        state = random.choice(list(regions_and_states[region].keys()))
        city = random.choice(regions_and_states[region][state])
        
        data.loc[i, 'dealerId'] = f"{(i + 1):06}"  # Dealer ID with 4 digits
        data.loc[i, 'dealerName'] = faker.unique.company()#f"{faker.first_name()} {faker.last_name()}"
        data.loc[i, 'passedTraining'] = random.choice(['Y']*7 + ['N']*3)
        data.loc[i, 'contactNumber'] = faker.msisdn()
        data.loc[i, 'country'] = 'Brazil'
        #data.loc[i, 'REGION'] = region
        data.loc[i, 'state'] = state
        data.loc[i, 'city'] = city
        data.loc[i, 'address'] = faker.address().replace('\n', ' ')  # Replace newline with space for better formatting

    return data

In [12]:
DEALER_INFO = generate_dealers(50)

In [13]:
DEALER_INFO

,dealerId,dealerName,passedTraining,contactNumber,country,state,city,address
0,000001,Alves S/A,N,5541999161510,Brazil,Paraná,Londrina,"Condomínio Mendes, 10 São Marcos 91413-145 Cir..."
1,000002,Aparecida,Y,5521963457923,Brazil,Ceará,Fortaleza,"Esplanada de Castro, 58 Candelaria 20769-845 R..."
2,000003,Marques e Filhos,Y,5521950842375,Brazil,Ceará,Fortaleza,"Trecho João Silva, 76 Palmares 09352337 Santos..."
3,000004,Guerra e Filhos,Y,5571902714278,Brazil,Ceará,Sobral,"Aeroporto de Andrade, 55 Anchieta 06381-206 Di..."
4,000005,Silva S/A,Y,5584991319344,Brazil,São Paulo,São Paulo,Passarela Enrico Almeida Ipe 14285-124 Apareci...
5,000006,Rodrigues da Luz - EI,Y,5561959097765,Brazil,Goiás,Anápolis,"Trecho Ana Albuquerque, 52 Lorena 51590-042 Pa..."
6,000007,Gonçalves,Y,5584924173042,Brazil,Pará,Ananindeua,"Loteamento Ana Clara Leão, 2 Serra Do Curral 7..."
7,000008,Cavalcante Pires - EI,Y,5571911133060,Brazil,Ceará,Fortaleza,"Rua de da Rocha, 47 Novo Aarão Reis 53492635 A..."
8,000009,Moura - ME,Y,5541917643039,Brazil,Goiás,Goiânia,Quadra Garcia Marieta 1ª Seção 19729668 Rocha ...
9,000010,Monteiro,Y,5541989305550,Brazil,Paraná,Curitiba,"Trevo Leonardo Garcia, 247 Virgínia 80132040 F..."


# Part Sale
Next we generate the *Part Sale* table, that contains all the Parts sold directly to the client

So based on the Problem that the vehicle has, the Dealer will sell the client a part
- Plan of Action
   - Map wich Parts can be sold for each specific problem.
   - Get a random number of how many different parts will be sold.
   - Then add a row for each part_sold.
 
   - Maior ou igual as partes utilizadas.

In [15]:
# Define parts and their descriptions with standardized 6-character codes
parts_type = {
    "BP0001": "Battery Pack",
    "EM0002": "Electric Motor",
    "IN0003": "Inverter",
    "CP0004": "Charging Port",
    "DC0005": "DC-DC Converter",
    "BMS006": "Battery Management System (BMS)",
    "PEC007": "Power Electronics Controller",
    "TMS008": "Thermal Management System",
    "RBS009": "Regenerative Braking System",
    "OC0100": "Onboard Charger",
    "HVC011": "High Voltage Cables",
    "EDU012": "Electric Drive unity",
    "RG0130": "Reduction Gear",
    "HP0140": "Heat Pump",
    "EAC015": "Electric Air Conditioning Compressor",
    "EWP016": "Electric Water Pump",
    "BP0170": "Brake Pads",
    "BD0180": "Brake Discs",
    "TI0190": "Tires",
    "WB0200": "Wheel Bearings",
    "WS0210": "Windshield",
    "MI0220": "Mirrors",
    "HL0230": "Headlights",
    "TL0240": "Taillights",
    "WB0250": "Wiper Blades",
    "SC0260": "Suspension Components (Shocks, Struts)",
    "CA0270": "Control Arms",
    "TRE028": "Tie Rod Ends",
    "CVJ029": "CV Joints",
    "AX0300": "Axles",
    "CL0310": "Coolant (for thermal management)",
    "BF0320": "Brake Fluid",
    "PSF033": "Power Steering Fluid (if applicable)",
    "CAF034": "Cabin Air Filter",
    "FB0350": "Fuse Box",
    "AB0360": "12V Auxiliary Battery"
}

# Define the maximum quantity for each part
max_partQuantity = {
    "BP0001": 1,  # Battery Pack
    "EM0002": 1,  # Electric Motor
    "IN0003": 1,  # Inverter
    "CP0004": 1,  # Charging Port
    "DC0005": 1,  # DC-DC Converter
    "BMS006": 1,  # Battery Management System (BMS)
    "PEC007": 1,  # Power Electronics Controller
    "TMS008": 1,  # Thermal Management System
    "RBS009": 1,  # Regenerative Braking System
    "OC0100": 1,  # Onboard Charger
    "HVC011": 4,  # High Voltage Cables
    "EDU012": 1,  # Electric Drive unity
    "RG0130": 1,  # Reduction Gear
    "HP0140": 1,  # Heat Pump
    "EAC015": 1,  # Electric Air Conditioning Compressor
    "EWP016": 1,  # Electric Water Pump
    "BP0170": 8,  # Brake Pads (Assuming a set for each wheel)
    "BD0180": 4,  # Brake Discs (Assuming a disc for each wheel)
    "TI0190": 4,  # Tires (Assuming a tire for each wheel)
    "WB0200": 4,  # Wheel Bearings (Assuming a bearing for each wheel)
    "WS0210": 1,  # Windshield
    "MI0220": 2,  # Mirrors
    "HL0230": 2,  # Headlights
    "TL0240": 2,  # Taillights
    "WB0250": 2,  # Wiper Blades
    "SC0260": 4,  # Suspension Components (Assuming components for each wheel)
    "CA0270": 2,  # Control Arms
    "TRE028": 2,  # Tie Rod Ends
    "CVJ029": 2,  # CV Joints
    "AX0300": 2,  # Axles
    "CL0310": 5,  # Coolant (Assuming 5 liters for a full system flush)
    "BF0320": 2,  # Brake Fluid (Assuming 2 liters for a full system flush)
    "PSF033": 2,  # Power Steering Fluid (if applicable)
    "CAF034": 1,  # Cabin Air Filter
    "FB0350": 1,  # Fuse Box
    "AB0360": 1   # 12V Auxiliary Battery
}


# Map problems to related parts using standardized codes
problem_parts_mapping = {
    "Battery degradation": ["BP0001", "BMS006"],
    "Charging issues": ["CP0004", "OC0100"],
    "Electric motor failure": ["EM0002"],
    "Range anxiety": ["BP0001"],  # Assuming range anxiety is related to battery pack
    "Battery management system failure": ["BMS006"],
    "Overheating of battery": ["BP0001", "TMS008"],
    "Electrical system faults": ["HVC011", "PEC007"],
    "Regenerative braking issues": ["RBS009"],
    "Inverter problems": ["IN0003"],
    "Cooling system issues": ["TMS008", "CL0310"],
    "Charging port issues": ["CP0004"],
    "Software glitches": ["PEC007"],
    "Wiring problems": ["HVC011"],
    "Vehicle control system failures": ["PEC007"],
    "Battery overheating": ["BP0001", "TMS008"],
    "Inconsistent charging": ["CP0004", "OC0100"],
    "Climate control system issues": ["EAC015", "HP0140"],
    "Connectivity issues": ["PEC007"],
    "Suspension problems": ["SC0260", "CA0270", "TRE028"],
    "Windshield replacement": ["WS0210"],
    "Brake system maintenance": ["BP0170", "BD0180", "BF0320"],
    "Tire replacement": ["TI0190"],
    "Fluid top-up": ["CL0310", "BF0320", "PSF033"],
    "Cabin filter replacement": ["CAF034"]
}

In [16]:
def generate_sale_parts(base_table):
    
    data = pd.DataFrame()

    t = 0
    i = 0
    
    additional_parts = []

    for i, row in base_table.iterrows():

        t = i

        createDate = faker.date_this_year()
        updateDate = faker.date_between(start_date=createDate)
        
        #Randomize the number of parts sold
        problem = row['CLAIM']
        parts_sold = randint(1,len(problem_parts_mapping[problem])) #problem_parts_mapping[problem] is the list of parts that could be sold for that problem

        #If more than one part is going to be sold, then we are going to store it in additional_parts and add it later, otherwise it is not possible
        #since we are iterating only once for each base_table row
        if parts_sold > 1:
            for x in range (1,parts_sold):
                additional_parts.append([problem_parts_mapping[problem][x],row['dealerOrderId'],vehicles[row['CAR']]])
            
        this_part = problem_parts_mapping[problem][0]
        partQuantity = randint(1,max_partQuantity[this_part])

        data.loc[i, 'roSalePartId'] = f"{(i):010}"
        data.loc[i, 'dealerOrderId'] = row['dealerOrderId']
        data.loc[i, 'partCode'] = f"PA{this_part}{vehicles[row['CAR']]}"
        #data.loc[i, 'partName'] = parts_type[this_part]
        data.loc[i, 'partQuantity'] = partQuantity
        data.loc[i, 'createBy'] = '9999999999'
        data.loc[i, 'createDate'] = createDate
        data.loc[i, 'updateBy'] = '9999999988'
        data.loc[i, 'updateDate'] = updateDate


    for i in range(len(additional_parts)):

         partQuantity = randint(1,max_partQuantity[additional_parts[i][0]])
        
         data.loc[i + t + 1, 'roSalePartId'] = f"{(t + (i + 1)):010}"
         data.loc[i + t + 1, 'dealerOrderId'] = additional_parts[i][1]
         data.loc[i + t + 1, 'partCode'] = f"PA{additional_parts[i][0]}{additional_parts[i][2]}"
         #data.loc[i + t + 1, 'partName'] = parts_type[additional_parts[i][0]]
         data.loc[i + t + 1, 'partQuantity'] = partQuantity
         data.loc[i + t + 1, 'createBy'] = '9999999999'
         data.loc[i + t + 1, 'createDate'] = createDate
         data.loc[i + t + 1, 'updateBy'] = '9999999988'
         data.loc[i + t + 1, 'updateDate'] = updateDate

    return data


In [17]:
REPAIR_ORDER_SALE_PARTS = generate_sale_parts(request_table)

In [18]:
REPAIR_ORDER_SALE_PARTS

,roSalePartId,dealerOrderId,partCode,partQuantity,createBy,createDate,updateBy,updateDate
1,0000000001,000001,PASC0260BYD004,3.0,9999999999,2024-01-21,9999999988,2024-07-16
2,0000000002,000002,PABMS006BYD001,1.0,9999999999,2024-05-25,9999999988,2024-06-02
3,0000000003,000003,PABMS006BYD001,1.0,9999999999,2024-06-06,9999999988,2024-07-08
4,0000000004,000004,PACP0004BYD001,1.0,9999999999,2024-03-31,9999999988,2024-07-07
5,0000000005,000005,PACP0004BYD001,1.0,9999999999,2024-06-25,9999999988,2024-06-26
6,0000000006,000005,PAOC0100BYD001,1.0,9999999999,2024-06-25,9999999988,2024-06-26


# Repair Part 
Next we generate the *Parts* used in the repair by the dealer.


In [20]:
def generate_parts(base_table):
    
    data = pd.DataFrame()

    
    additional_parts = []

    for i, row in base_table.iterrows():

        data.loc[i, 'roPartId '] = f"{(i):010}"
        data.loc[i, 'dealerOrderId'] = row['dealerOrderId']
        data.loc[i, 'partCode'] = row['partCode']
        #data.loc[i, 'partName'] = row['partCode']
        data.loc[i, 'partQuantity'] = row['partQuantity']
        data.loc[i, 'mainPartFlag'] = f"{randint(1000,1000000):08}"
        data.loc[i, 'downPartSerialNumber'] = None
        data.loc[i, 'upPartSerialNumber'] = None
        data.loc[i, 'createBy'] = '9999999999'
        data.loc[i, 'createDate'] = row['createDate']
        data.loc[i, 'updateBy'] = '9999999988'
        data.loc[i, 'updateDate'] = faker.date_between(start_date= row['createDate'])

    return data


In [21]:
REPAIR_ORDER_PARTS = generate_parts(REPAIR_ORDER_SALE_PARTS)

In [22]:
REPAIR_ORDER_PARTS

,roPartId,dealerOrderId,partCode,partQuantity,mainPartFlag,downPartSerialNumber,upPartSerialNumber,createBy,createDate,updateBy,updateDate
1,0000000001,000001,PASC0260BYD004,3.0,00098908,None,None,9999999999,2024-01-21,9999999988,2024-04-23
2,0000000002,000002,PABMS006BYD001,1.0,00737716,None,None,9999999999,2024-05-25,9999999988,2024-07-24
3,0000000003,000003,PABMS006BYD001,1.0,00299385,None,None,9999999999,2024-06-06,9999999988,2024-06-15
4,0000000004,000004,PACP0004BYD001,1.0,00019350,None,None,9999999999,2024-03-31,9999999988,2024-05-22
5,0000000005,000005,PACP0004BYD001,1.0,00684267,None,None,9999999999,2024-06-25,9999999988,2024-07-19
6,0000000006,000005,PAOC0100BYD001,1.0,00349350,None,None,9999999999,2024-06-25,9999999988,2024-07-18


# Parts Info Table

- For each Part in the parts dict
   - For each car model in the vehicle dict
       - there will be a partCode

In [24]:
def generate_PARTS_INFO():
    
    data = []
    
    for vehicle_name, vehicle_code in vehicles.items():
        for partCode, partName in parts_type.items():
            partCode_vehicle = f"PA{partCode}{vehicle_code}"
            data.append({
                "partCode": partCode_vehicle,
                "partName": partName,
                "unity": "Piece",  # Assuming unity is piece for simplicity
                "isPermittedToOrder": random.choice(["Yes", "No"]),
                "permiteedOrderType": random.choice(["Type1", "Type2"]),
                "sellingPrice": round(random.uniform(10, 1000), 2),
                "suggestedRetailPrice": round(random.uniform(20, 1200), 2),
                "warrantyPrice": round(random.uniform(5, 900), 2),
                "vehicleSeries": vehicle_name,
                "volume": round(random.uniform(0.1, 10), 2),
                "region": random.choice(list(regions_and_states.keys())),
                "sparePartsType": "OEM",
                "isOemParts": "Yes",
                "creationDate": faker.date_this_year()
            })
    
    df = pd.DataFrame(data)
    return df

In [25]:
PARTS_INFO = generate_PARTS_INFO()

In [26]:
PARTS_INFO

,partCode,partName,unity,isPermittedToOrder,permiteedOrderType,sellingPrice,suggestedRetailPrice,warrantyPrice,vehicleSeries,volume,region,sparePartsType,isOemParts,creationDate
0,PABP0001BYD001,Battery Pack,Piece,No,Type2,786.05,387.74,643.66,Dolphin,9.05,Brazil Central-West,OEM,Yes,2024-06-02
1,PAEM0002BYD001,Electric Motor,Piece,No,Type1,258.47,83.64,292.85,Dolphin,8.80,Brazil South,OEM,Yes,2024-07-02
2,PAIN0003BYD001,Inverter,Piece,No,Type1,925.17,777.23,66.66,Dolphin,2.11,Brazil Northeast,OEM,Yes,2024-04-13
3,PACP0004BYD001,Charging Port,Piece,Yes,Type2,912.81,880.42,732.08,Dolphin,6.16,Brazil Southeast,OEM,Yes,2024-03-11
4,PADC0005BYD001,DC-DC Converter,Piece,Yes,Type1,435.72,288.75,491.28,Dolphin,0.34,Brazil North,OEM,Yes,2024-03-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,PABF0320BYD005,Brake Fluid,Piece,Yes,Type2,568.82,149.67,602.89,Tan,8.75,Brazil Northeast,OEM,Yes,2024-02-18
176,PAPSF033BYD005,Power Steering Fluid (if applicable),Piece,Yes,Type2,896.67,31.84,291.09,Tan,8.81,Brazil Southeast,OEM,Yes,2024-07-16
177,PACAF034BYD005,Cabin Air Filter,Piece,Yes,Type2,944.72,658.83,46.56,Tan,8.09,Brazil Southeast,OEM,Yes,2024-06-22
178,PAFB0350BYD005,Fuse Box,Piece,No,Type2,864.04,419.06,230.55,Tan,5.35,Brazil South,OEM,Yes,2024-07-01


# Labours Table

This table contains the Labours and their cost to the company

In [28]:
labours = {
    "LR001": "Battery Replacement",
    "LR002": "Motor Repair",
    "LR003": "Inverter Replacement",
    "LR004": "Charging Port Repair",
    "LR005": "DC-DC Converter Replacement",
    "LR006": "BMS Calibration",
    "LR007": "Power Electronics Repair",
    "LR008": "Thermal Management Service",
    "LR009": "Regenerative Braking Adjustment",
    "LR010": "Onboard Charger Repair",
    "LR011": "Drive unity Maintenance",
    "LR012": "Gearbox Service",
    "LR013": "Heat Pump Repair",
    "LR014": "AC Compressor Service",
    "LR015": "Water Pump Replacement",
    "LR016": "Brake System Service",
    "LR017": "Suspension Repair",
    "LR018": "Headlight Alignment",
    "LR019": "Windshield Replacement",
    "LR020": "Fuse Replacement"
}

labours_category = {
    "LR001": {"Primary": "Electrical", "Secondary": "Battery"},
    "LR002": {"Primary": "Mechanical", "Secondary": "Motor"},
    "LR003": {"Primary": "Electrical", "Secondary": "Inverter"},
    "LR004": {"Primary": "Electrical", "Secondary": "Charging Port"},
    "LR005": {"Primary": "Electrical", "Secondary": "DC-DC Converter"},
    "LR006": {"Primary": "Electrical", "Secondary": "BMS"},
    "LR007": {"Primary": "Electrical", "Secondary": "Power Electronics"},
    "LR008": {"Primary": "Thermal Management", "Secondary": "Thermal Management"},
    "LR009": {"Primary": "Electrical", "Secondary": "Regenerative Braking"},
    "LR010": {"Primary": "Electrical", "Secondary": "Onboard Charger"},
    "LR011": {"Primary": "Mechanical", "Secondary": "Drive Unit"},
    "LR012": {"Primary": "Mechanical", "Secondary": "Gearbox"},
    "LR013": {"Primary": "Thermal Management", "Secondary": "Heat Pump"},
    "LR014": {"Primary": "Thermal Management", "Secondary": "AC Compressor"},
    "LR015": {"Primary": "Cooling System", "Secondary": "Water Pump"},
    "LR016": {"Primary": "Brake System", "Secondary": "Brake System"},
    "LR017": {"Primary": "Suspension", "Secondary": "Suspension"},
    "LR018": {"Primary": "Lighting", "Secondary": "Headlight"},
    "LR019": {"Primary": "Body", "Secondary": "Windshield"},
    "LR020": {"Primary": "Electrical", "Secondary": "Fuse"}
}
parts_to_labours_mapping = {
    "BP0001": "LR001",
    "EM0002": "LR002",
    "IN0003": "LR003",
    "CP0004": "LR004",
    "DC0005": "LR005",
    "BMS006": "LR006",
    "PEC007": "LR007",
    "TMS008": "LR008",
    "RBS009": "LR009",
    "OC0100": "LR010",
    "HVC011": "LR010",
    "EDU012": "LR011",
    "RG0130": "LR012",
    "HP0140": "LR013",
    "EAC015": "LR014",
    "EWP016": "LR015",
    "BP0170": "LR016",
    "BD0180": "LR016",
    "TI0190": "LR016",
    "WB0200": "LR016",
    "WS0210": "LR019",
    "MI0220": "LR019",
    "HL0230": "LR018",
    "TL0240": "LR018",
    "WB0250": "LR018",
    "SC0260": "LR017",
    "CA0270": "LR017",
    "TRE028": "LR017",
    "CVJ029": "LR017",
    "AX0300": "LR017",
    "CL0310": "LR013",
    "BF0320": "LR013",
    "PSF033": "LR013",
    "CAF034": "LR014",
    "FB0350": "LR020",
    "AB0360": "LR020"
}

labours_extra = {                       #Labours that are not directly linked to parts
    "LR021": "Software Update",
    "LR022": "Diagnostic Check",
    "LR023": "Annual Maintenance",
    "LR024": "Battery Health Check",
    "LR025": "General Safety Inspection"
}

In [29]:
def generate_labours(parts_data):

    data = pd.DataFrame()
    
    for index, row in parts_data.iterrows():
        
        t = index                            #0 1 2 3 4 5 6 7 8 9 10 11 12 13
        partCode = (row['partCode'])[2:8]  #P A B P 0 0 0 1 B Y D  0  0  1
        labour_code = f"W{parts_to_labours_mapping[partCode]}{(row['partCode'])[8:14]}" #WLR001BYD002
        labour_name = parts_to_labours_mapping[partCode]

        createDate = faker.date_between(row['createDate']) #Date of the labour (between the selling of parts and now days)

        data.loc[index,'roLabourId'] = index + 1
        data.loc[index,'dealerOrderId'] = row['dealerOrderId']
        data.loc[index,'labourCode'] = labour_code
        #data.loc[index,'labourName'] = labour_name
        data.loc[index,'labourHours'] = randint(1,6)
        data.loc[index,'createBy'] = '9999999999'
        data.loc[index,'createDate'] = createDate
        data.loc[index,'updateBy'] = '9999999988'
        data.loc[index,'updateDate'] = faker.date_between(createDate)

    return data
    
    # for i in range (t, t + t//3):    #Suppose 1/3 of the orders need some of the labours_extra

    #     labour = random.choice(list(labours_extra.keys()))
    #     labour_name = labours_extra[labour]
    #     labour_code = f"W{labour}{vehicles[request_table.loc[row['dealerOrderId']]]}"

    #     createDate = faker.date_this_year()
        
    #     data.loc[i,'roLabourId'] = i
    #     data.loc[i,'dealerOrderId'] = randint(1,5000)
    #     data.loc[i,'labourCode'] = labour_code
    #     data.loc[i,'labourName'] = labour_name
    #     data.loc[i,'createBy'] = '9999999999'
    #     data.loc[i,'createDate'] = createDate
    #     data.loc[i,'updateBy'] = '9999999988'
    #     data.loc[i,'updateDate'] = faker.date_between(createDate)
        
    # return data


In [30]:
REPAIR_ORDER_LABOUR = generate_labours(REPAIR_ORDER_SALE_PARTS)

In [31]:
REPAIR_ORDER_LABOUR

,roLabourId,dealerOrderId,labourCode,labourHours,createBy,createDate,updateBy,updateDate
1,2.0,000001,WLR017BYD004,5.0,9999999999,2024-04-18,9999999988,2024-06-22
2,3.0,000002,WLR006BYD001,3.0,9999999999,2024-06-06,9999999988,2024-07-16
3,4.0,000003,WLR006BYD001,4.0,9999999999,2024-06-30,9999999988,2024-07-18
4,5.0,000004,WLR004BYD001,2.0,9999999999,2024-04-15,9999999988,2024-07-22
5,6.0,000005,WLR004BYD001,1.0,9999999999,2024-07-11,9999999988,2024-07-16
6,7.0,000005,WLR010BYD001,6.0,9999999999,2024-07-17,9999999988,2024-07-20


# Labours Table

This table contains the Labours and their cost to the company

In [33]:
def generate_LABOUR_INFO_table():    
    data = []
    
    for vehicle_name, vehicle_code in vehicles.items():
        for labour_code, labour_name in labours.items():

            if labour_code in ["LR001","LR006"]:
                battery_pack_repair = 'Y'
            else:
                battery_pack_repair = 'N'
            
            labour_code_vehicle = f"W{labour_code}{vehicle_code}"
            data.append({
                "labourCode": labour_code_vehicle,
                "labourName": f"{labour_name}",
                "vehicleSeries": vehicle_name,
                "mainCategory": labours_category.get(labour_code, {}).get("Primary", "Labour code not found"),
                "secondaryClassification":labours_category.get(labour_code, {}).get("Secondary", "Labour code not found"),
                "isBatteryRepair":battery_pack_repair
            })
    
    df = pd.DataFrame(data)
    return df


# labourCode
# labourName (Repair item name)
# vehicleSeries (Name of project Vehicle Series)
# mainCategory (Main category name)
# secondaryClassification (Secondary classification name)
# IS_BATTERY_REPAIR (Battery pack repair or not)

In [34]:
LABOUR_INFO = generate_LABOUR_INFO_table()

In [35]:
LABOUR_INFO

,labourCode,labourName,vehicleSeries,mainCategory,secondaryClassification,isBatteryRepair
0,WLR001BYD001,Battery Replacement,Dolphin,Electrical,Battery,Y
1,WLR002BYD001,Motor Repair,Dolphin,Mechanical,Motor,N
2,WLR003BYD001,Inverter Replacement,Dolphin,Electrical,Inverter,N
3,WLR004BYD001,Charging Port Repair,Dolphin,Electrical,Charging Port,N
4,WLR005BYD001,DC-DC Converter Replacement,Dolphin,Electrical,DC-DC Converter,N
...,...,...,...,...,...,...
95,WLR016BYD005,Brake System Service,Tan,Brake System,Brake System,N
96,WLR017BYD005,Suspension Repair,Tan,Suspension,Suspension,N
97,WLR018BYD005,Headlight Alignment,Tan,Lighting,Headlight,N
98,WLR019BYD005,Windshield Replacement,Tan,Body,Windshield,N


In [36]:
REQUEST_TABLE = create_request_table(5000)
REPAIR_ORDER_MAIN = generate_main_table(REQUEST_TABLE)
DEALER_INFO = generate_dealers(50)
REPAIR_ORDER_SALE_PARTS = generate_sale_parts(REQUEST_TABLE)
REPAIR_ORDER_PARTS = generate_parts(REPAIR_ORDER_SALE_PARTS)
PARTS_INFO = generate_PARTS_INFO()
REPAIR_ORDER_LABOUR = generate_labours(REPAIR_ORDER_SALE_PARTS)
LABOUR_INFO = generate_LABOUR_INFO_table()

In [37]:
import os
directory = 'data_files'
os.makedirs(directory, exist_ok=True)

REQUEST_TABLE.to_csv(os.path.join(directory, 'request_table.csv'), index=False)
REPAIR_ORDER_MAIN.to_csv(os.path.join(directory, 'repair_order_main.csv'), index=False)
DEALER_INFO.to_csv(os.path.join(directory, 'dealer_info.csv'), index=False)
REPAIR_ORDER_SALE_PARTS.to_csv(os.path.join(directory, 'repair_order_sale_parts.csv'), index=False)
REPAIR_ORDER_PARTS.to_csv(os.path.join(directory, 'repair_order_parts.csv'), index=False)
PARTS_INFO.to_csv(os.path.join(directory, 'parts_info.csv'), index=False)
REPAIR_ORDER_LABOUR.to_csv(os.path.join(directory, 'repair_order_labour.csv'), index=False)
LABOUR_INFO.to_csv(os.path.join(directory, 'labour_info.csv'), index=False)